## Notebook 1

Think about the goals of Project 3 in terms of **supervised learning** and **unsupervised learning**. Use this to start your report!

In [1]:
!conda install psycopg2 --yes  ###unneeded if already downloaded onto container


Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    krb5:     1.14.2-0       conda-forge
    libpq:    9.6.3-0        conda-forge
    psycopg2: 2.7.3.1-py36_0 conda-forge
    pyqt:     5.6.0-py36_4   conda-forge
    qt:       5.6.2-3        conda-forge

The following packages will be DOWNGRADED:

    numexpr:  2.6.4-py36_0   conda-forge --> 2.6.2-np112py36_nomkl_0 defaults [nomkl]

krb5-1.14.2-0. 100% |################################| Time: 0:00:01   2.24 MB/s
libpq-9.6.3-0. 100% |################################| Time: 0:00:00 612.91 kB/s
psycopg2-2.7.3 100% |################################| Time: 0:00:00 902.86 kB/s
numexpr-2.6.2- 100% |################################| Time: 0:00:00   1.24 MB/s
qt-5.6.2-3.tar 100% |################################| Time: 0:00:08   5.67 MB/s
pyqt-5.6.0-py3 100% |################################| Time: 0:00:01   4.92 MB/s

In [1]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

In [1]:
import pandas as pd

In [14]:
import csv
from IPython.display import display
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Connecting to the Database

#### 1. Create your connection

In [ ]:
con = pg2.connect(host='34.211.227.227',   ###credentials for the Proj3
                  dbname='postgres',
                  user='postgres')

#### 2. Create a cursor from that connection

In [17]:
cur = con.cursor(cursor_factory=RealDictCursor)  ###

#### 3. Execute a query over the cursor

In [18]:
cur.execute('SELECT * FROM madelon LIMIT 2;') 

#### 4. Use the cursor to fetch the results

In [19]:
results = cur.fetchall()

#### 5. Close the connection

In [20]:
con.close()  #very important, must close connection.  We are all using the same DB and each connection uses memory

#### 6. Display the results

In [21]:
results  ###results is a simple dictionary

[{'_id': 166512,
  'feat_000': 0.694155714303468,
  'feat_001': -1.42025132263353,
  'feat_002': 0.234614160225966,
  'feat_003': -0.505374813947096,
  'feat_004': -1.0796866179213,
  'feat_005': -1.66311024311938,
  'feat_006': -0.362041382924994,
  'feat_007': 1.65339927521212,
  'feat_008': 1.52630320346286,
  'feat_009': 0.592263796811647,
  'feat_010': -0.16986729708039,
  'feat_011': 1.78509713504284,
  'feat_012': 1.45002586741219,
  'feat_013': 0.543047746145322,
  'feat_014': 1.07879536236884,
  'feat_015': 0.735958365817003,
  'feat_016': 0.130319246541972,
  'feat_017': -1.6749389638547,
  'feat_018': 1.32025770142309,
  'feat_019': 0.753388374116329,
  'feat_020': -0.361083913660042,
  'feat_021': -0.757041971499449,
  'feat_022': 0.394769846727542,
  'feat_023': 1.75388167346691,
  'feat_024': -0.822390596999948,
  'feat_025': 0.660766868690795,
  'feat_026': -0.633412257559098,
  'feat_027': -0.168962655616009,
  'feat_028': -0.52112110034726,
  'feat_029': -0.52317307874

In [23]:
df = pd.DataFrame(results)
df

,_id,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_991,feat_992,feat_993,feat_994,feat_995,feat_996,feat_997,feat_998,feat_999,target
0,166512,0.694156,-1.420251,0.234614,-0.505375,-1.079687,-1.663110,-0.362041,1.653399,1.526303,...,-0.128677,-0.613816,-0.874937,-0.169035,1.905562,-0.368597,-1.261831,1.162328,-0.965722,0
1,166513,-2.031752,1.572075,-0.030956,0.210886,-0.672458,-0.133269,0.278524,0.151338,1.263312,...,-0.584206,0.226024,-0.681836,0.229200,-0.774399,-1.094383,-0.708891,-1.057284,-0.940768,1


#### 6. Confirming the length of the dataset.
### 200,000 from Madelon set
 10% of 200,000 = 20,000 from the Madelon set (from instructors)

### From the UCI website , summary totals below
MADELON -- Positive ex. -- Negative ex. -- Total

Training set -- 1000 -- 1000 -- 2000

Validation set -- 300 -- 300 -- 600

Test set -- 900 -- 900 -- 1800

All -- 2200 -- 2200 -- 4400

Number of variables/features/attributes:
Real: 20
Probes: 480
Total: 500 

In [31]:
# con = pg2.connect(host='34.211.227.227',
#                   dbname='postgres',
#                   user='postgres')
# cur = con.cursor(cursor_factory=RealDictCursor)
# cur.execute('SELECT COUNT(*) FROM madelon;')   
# results = cur.fetchall()
# con.close()

In [32]:
results   ###just to confirm the length of the Madelon dataset

[{'count': 200000}]

### Function for fetching results - Prefered Method

In [1]:
def con_cur_to_class_db():  ###best practice, since we will run ut over and over
    con = pg2.connect(host='34.211.227.227',
                  dbname='postgres',
                  user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

In [5]:
### BE VERY CAREFUL, MAKE SURE YOU ARE ON AWS MEDIUM, NOT T2.MICRO


# con, cur = con_cur_to_class_db()
# cur.execute('SELECT * FROM madelon LIMIT 20000;')  ###test @ 200
# results = cur.fetchall()
# con.close()

In [6]:
# madelonDB_df_3 = pd.DataFrame(results)
madelonDB_df_3.head()

,_id,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_991,feat_992,feat_993,feat_994,feat_995,feat_996,feat_997,feat_998,feat_999,target
0,65728,-1.543619,0.128881,-0.213664,-0.515320,1.387451,1.195127,0.935323,0.012586,-0.443595,...,-0.302952,-0.465247,0.642401,-1.611977,0.821679,0.547925,-0.009201,-1.059493,-1.602108,0
1,65729,-0.109869,0.350016,0.198801,-1.124599,1.826685,-0.959312,-1.694781,0.324573,-0.413039,...,0.263585,-0.366926,0.163896,0.083760,-0.914880,0.373018,-0.548678,-1.306312,2.174954,1
2,65730,3.858771,-1.710262,-1.244474,-0.788795,0.186988,0.522353,-0.613506,-0.386157,-0.583601,...,1.897503,-1.715621,0.465021,0.250617,-0.714833,1.054906,1.090022,-1.216019,-0.039856,0
3,65731,1.520146,0.010613,-1.838139,0.785676,0.582298,0.121762,0.540609,0.666417,1.065765,...,0.153788,0.802806,1.087427,0.935520,0.424076,0.680056,-0.386010,0.501438,-3.261500,1
4,65732,-1.056974,1.330800,-1.437783,1.809255,-0.074225,0.578842,-0.172347,0.884177,-0.125768,...,0.597338,0.380561,0.429402,-1.337828,0.755074,-1.371980,-0.390992,-1.318051,0.053820,0


In [7]:
madelonDB_df_3.shape

(20000, 1002)

In [17]:
# madelonDB_df_1.to_pickle('../pickle_jar/madelonDB_df_1.p')

In [21]:
# madelonDB_df_2.to_pickle('../pickle_jar/madelonDB_df_2.p')

In [8]:
# madelonDB_df_3.to_pickle('../pickle_jar/madelonDB_df_3.p')

### EDA

In [10]:
madelonDB_df_1 = pd.read_pickle('../pickle_jar/madelonDB_df_1.p')

In [ ]:
# madelonDB_df_2 = pd.read_pickle('../pickle_jar/madelonDB_df_2.p')

In [ ]:
# madelonDB_df_3 = pd.read_pickle('../pickle_jar/madelonDB_df_3.p')

In [11]:
# madelonDB_df.dtypes

In [12]:
madelonDB_df_1.shape

(20000, 1002)

In [14]:
madelonDB_df_1.isnull().any().sum()  ###no NULL values!!

0

In [2]:
DB_madelon = pd.read_pickle('../pickle_jar/madelon_df_sample1.p')
DB_madelon.shape

(6506, 1000)

In [3]:
target_df = pd.read_pickle('../pickle_jar/DB_target.p')
target_df.shape

(6506,)

In [7]:
target_df.head()

0    1
1    1
2    1
3    1
4    0
Name: target, dtype: int64

 ### Benchmarking

In [8]:
###Careful, running this will empty the results list!!!

benchmarking_results = []  #empty list

def benchmarking(model, scaled=False):  ###X and a column in X
    ### X is 2000 rows x 500 features
    ### y is 2000 rows x 1 feature
    X = DB_madelon
    y = target_df
    
    X_train, \
    X_test, \
    y_train,  \
    y_test = train_test_split(X, y, test_size=0.25)
    
    if scaled:
        ss = StandardScaler()
        X_train_scaled = ss.fit_transform(X_train)
        X_test_scaled = ss.transform(X_test)
        
        classifier = model()
        classifier.fit(X_train_scaled, y_train)
    
        score = classifier.score(X_test_scaled, y_test)   
    else:   
    
        classifier = model()
        classifier.fit(X_train, y_train)
    
        score = classifier.score(X_test, y_test)
    
    benchmarking_results.append({model:score})
    return score

### Benchmarking with K-Neighbors Classifier

In [9]:
benchmarking(KNeighborsClassifier)

0.48678549477566074

In [10]:
benchmarking_results

[{sklearn.neighbors.classification.KNeighborsClassifier: 0.48678549477566074}]

### Benchmarking with LogisticRegression

In [11]:
benchmarking(LogisticRegression)

0.50522433927473875

In [12]:
benchmarking_results

[{sklearn.neighbors.classification.KNeighborsClassifier: 0.48678549477566074},
 {sklearn.linear_model.logistic.LogisticRegression: 0.50522433927473875}]

### Benchmarking with Decision_tree

In [13]:
benchmarking(DecisionTreeClassifier)

0.4990780577750461

In [28]:
benchmarking_results

[{sklearn.tree.tree.DecisionTreeClassifier: 0.6784},
 {sklearn.linear_model.logistic.LogisticRegression: 0.56279999999999997},
 {sklearn.tree.tree.DecisionTreeClassifier: 0.68620000000000003}]

In [29]:
DB_benchmarking_results_df = pd.Series(benchmarking_results)
DB_benchmarking_results_df

0    {<class 'sklearn.tree.tree.DecisionTreeClassif...
1    {<class 'sklearn.linear_model.logistic.Logisti...
2    {<class 'sklearn.tree.tree.DecisionTreeClassif...
dtype: object

In [30]:
DB_benchmarking_results_df.to_pickle('../pickle_jar/DB_benchmarking_results_df.p')

In [31]:
DB_benchmarking_results_df = pd.read_pickle('../pickle_jar/DB_benchmarking_results_df.p')

### Benchmarking with SVC

In [15]:
benchmarking(SVC)

0.49539028887523051

In [16]:
benchmarking_results

[{sklearn.neighbors.classification.KNeighborsClassifier: 0.48678549477566074},
 {sklearn.linear_model.logistic.LogisticRegression: 0.50522433927473875},
 {sklearn.tree.tree.DecisionTreeClassifier: 0.4990780577750461},
 {sklearn.svm.classes.SVC: 0.49539028887523051}]

### Preliminary benchmarking results

 - Logistic Regression Benchmark    : 0.50
 - KNN Classifier : 0.49 
 - Decision Tree Classifier : 0.50
 - Support Vector Classifier : 0.49